In [1]:
import pyvista as pv

In [23]:

import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d.axes3d import Axes3D
from BabelViscoFDTD.tools.RayleighAndBHTE import GenerateFocusTx,SpeedofSoundWater, GenerateSurface
import os


extlay={}
TemperatureWater=37.0
extlay['c']=SpeedofSoundWater(TemperatureWater)



def computeExaNeuroGeometry():
    # transxyz = np.loadtxt(os.path.join(os.path.dirname(os.path.realpath(__file__)),'ExaNeuroTransducerGeometry.csv'),delimiter=',',skiprows=0)
    transxyz = np.loadtxt(os.path.join('.','ExaNeuroTransducerGeometry.csv'),delimiter=',',skiprows=0)
    assert(transxyz.shape[0]==1024) #number of elements
    assert(transxyz.shape[1]==4) #element, X,Y,Z coordinates in mm, and area in mm2
    transxyz=transxyz[:,:3] #we skip the Tx element coordinates only
    return transxyz*1e-3

DimensionElem = np.sqrt(113.54)*1e-3
DimensionElem = 10e-3
FREQ=650e3

def GenerateSingleElem(PPW=12.0,Foc=150e-3):
    #60.08 PPW produces close to integer steps for both pitch and kerf
    
    Tx = {}

    step = 1500/FREQ/PPW
    hstep=step/2.0
    latSteps=int(np.round(DimensionElem/step))
    print('latSteps',latSteps)
    step=DimensionElem/latSteps

    centersX= np.arange(latSteps)*step
    centersX-=np.mean(centersX)

    SingElem = np.zeros((latSteps**2,3))
    N = np.zeros((SingElem.shape[0],3))
    N[:,2]=1
    ds = np.ones((SingElem.shape[0],1))*step**2

    VertDisplay=  np.zeros((SingElem.shape[0]*4,3))
    FaceDisplay= np.arange(SingElem.shape[0]*4,dtype=int).reshape((SingElem.shape[0],4))

    XX,YY=np.meshgrid(centersX,centersX)
    SingElem[:,0]=XX.flatten()
    SingElem[:,1]=YY.flatten()
    SingElem[:,2]=-Foc

    VertDisplay[0::4,0]=SingElem[:,0]-hstep
    VertDisplay[0::4,1]=SingElem[:,1]-hstep
    
    VertDisplay[1::4,0]=SingElem[:,0]+hstep
    VertDisplay[1::4,1]=SingElem[:,1]-hstep

    VertDisplay[2::4,0]=SingElem[:,0]+hstep
    VertDisplay[2::4,1]=SingElem[:,1]+hstep

    VertDisplay[3::4,0]=SingElem[:,0]-hstep
    VertDisplay[3::4,1]=SingElem[:,1]+hstep

    VertDisplay[:,2]=-Foc

   
    Tx['center'] = SingElem 
    Tx['ds'] = ds
    Tx['normal'] = N
    Tx['VertDisplay'] = VertDisplay 
    Tx['FaceDisplay'] = FaceDisplay 
    return Tx

def GenerateExaNeuroTx(Frequency=650e3,RotationZ=0,FactorEnlarge=1):

    f=Frequency;
    Foc=150e-3*FactorEnlarge
    Diameter=10e-3*FactorEnlarge

    #%This is the indiv tx element
    # TxElem=GenerateSingleElem()
    TxElem=GenerateFocusTx(f,Foc,Diameter,extlay['c'],PPWSurface=12)

    transLoc = computeExaNeuroGeometry()
    # transLoc=transLoc[[0,1023],:]
  
    transLocDisplacedZ=transLoc.copy()
    transLocDisplacedZ[:,2]-=Foc
    # print(transLocDisplacedZ)
    NElems=transLoc.shape[0]
    nSubElems=len( TxElem['ds'])
   
    nSubElemsVert=TxElem['VertDisplay'].shape[0]
    # print('nSubElemsVert',nSubElemsVert)

    XYNorm=np.linalg.norm(transLocDisplacedZ[:,:2],axis=1)
    VN=np.linalg.norm(transLocDisplacedZ,axis=1)
    theta=np.arcsin(XYNorm/VN)
    phi=np.arctan2(transLocDisplacedZ[:,1],transLocDisplacedZ[:,0])
    phi+=np.deg2rad(RotationZ)
    # print('theta',theta)
    # print('phi',phi)

    TxExaNeuro={}
    TxExaNeuro['center'] = np.zeros((nSubElems*NElems,3))
    TxExaNeuro['elemcenter'] = np.zeros((len(theta),3))
    TxExaNeuro['ds'] = np.zeros((nSubElems*NElems,1))
    TxExaNeuro['normal'] = np.zeros((nSubElems*NElems,3))
    TxExaNeuro['elemdims']=TxElem['ds'].size
    TxExaNeuro['NumberElems']=len(theta)
    TxExaNeuro['VertDisplay'] = np.zeros((nSubElemsVert*NElems,3))
    TxExaNeuro['FaceDisplay'] = np.zeros((nSubElems*NElems,4),np.int64)

    for n in range(len(theta)):
        rotateMatrixY = np.array([[np.cos(theta[n]),0,np.sin(theta[n])],[0,1,0],[-np.sin(theta[n]),0,np.cos(theta[n])]])
        rotateMatrixZ = np.array([[-np.cos(phi[n]),np.sin(phi[n]),0],[-np.sin(phi[n]),-np.cos(phi[n]),0],[0,0,1]])
        rotateMatrix = rotateMatrixZ@rotateMatrixY
       
        center=(rotateMatrix@TxElem['center'].T).T
        TxExaNeuro['elemcenter'][n,:]=np.mean(center,axis=0) # the very first subelement is at the center
        
        normal=(rotateMatrix@TxElem['normal'].T).T

        VertDisplay=(rotateMatrix@TxElem['VertDisplay'].T).T
       
        TxExaNeuro['center'][n*nSubElems:(n+1)*nSubElems,:]=center
        TxExaNeuro['ds'][n*nSubElems:(n+1)*nSubElems]=TxElem['ds']
        TxExaNeuro['normal'][n*nSubElems:(n+1)*nSubElems,:]=normal
        TxExaNeuro['VertDisplay'][n*nSubElemsVert:(n+1)*nSubElemsVert,:]=VertDisplay
        TxExaNeuro['FaceDisplay'][n*nSubElems:(n+1)*nSubElems,:]=TxElem['FaceDisplay']+n*nSubElemsVert

    TxExaNeuro['VertDisplay'][:,2]+=Foc
    TxExaNeuro['center'][:,2]+=Foc
    TxExaNeuro['elemcenter'][:,2]+=Foc
    
    print('Aperture dimensions (x,y) =',TxExaNeuro['center'][:,0].max()-TxExaNeuro['center'][:,0].min(),
                                        TxExaNeuro['center'][:,1].max()-TxExaNeuro['center'][:,1].min())

    print('Aperture dimensions (x,y) =',TxExaNeuro['VertDisplay'][:,0].max()-TxExaNeuro['VertDisplay'][:,0].min(),
                                        TxExaNeuro['VertDisplay'][:,1].max()-TxExaNeuro['VertDisplay'][:,1].min())


    TxExaNeuro['FocalLength']=Foc
    TxExaNeuro['Aperture']=np.max([TxExaNeuro['VertDisplay'][:,0].max()-TxExaNeuro['VertDisplay'][:,0].min(),
                                      TxExaNeuro['VertDisplay'][:,1].max()-TxExaNeuro['VertDisplay'][:,1].min()]);
    return TxExaNeuro


In [24]:
np.sqrt(113.54/np.pi)*2

12.023461145162418

In [25]:
Tx=GenerateExaNeuroTx()
Tx['center'].max(axis=0)

Aperture dimensions (x,y) = 0.29976906287303673 0.2998633996913339
Aperture dimensions (x,y) = 0.2997765905658488 0.2998682126338254


array([0.14987142, 0.14986851, 0.14901328])

In [26]:

Faces=np.hstack((np.ones((Tx['FaceDisplay'].shape[0],1),int)*4,Tx['FaceDisplay'])) 
mesh = pv.PolyData(Tx['VertDisplay'],Faces)
# Plot using the ITKplotter
pv.plot(mesh,color='r')

Widget(value='<iframe src="http://localhost:59380/index.html?ui=P_0x32f283c70_5&reconnect=auto" class="pyvista…

In [142]:
from stl import mesh
#we also export the STL of the Tx for display in Brainsight or 3D slicer
TxVert=Tx['VertDisplay'].T.copy()

TxStl = mesh.Mesh(np.zeros(Tx['FaceDisplay'].shape[0]*2, dtype=mesh.Mesh.dtype))

TxVert=TxVert.T[:,:3]
for i, f in enumerate(Tx['FaceDisplay']):
    TxStl.vectors[i*2][0] = TxVert[f[0],:]
    TxStl.vectors[i*2][1] = TxVert[f[1],:]
    TxStl.vectors[i*2][2] = TxVert[f[3],:]

    TxStl.vectors[i*2+1][0] = TxVert[f[1],:]
    TxStl.vectors[i*2+1][1] = TxVert[f[2],:]
    TxStl.vectors[i*2+1][2] = TxVert[f[3],:]

TxStl.save('Tx.stl')